Resultados esperados:
 - input: 
    - avg_runtime
    - input_complexity
    - speedup
    - eficiencia
 - output: 
    - p_cores


In [1]:
import pandas as pd
import tensorflow as tf
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import mean_squared_error

In [2]:
df_clean = pd.read_csv("dataset/clean_data.csv")

df_clean = df_clean.round(decimals=2)

# df_clean['cores'] = df_clean['cores']/df_clean['cores'].max()
df_clean['input'] = df_clean['input']/df_clean['input'].max()

In [4]:
df_clean.columns

Index(['cores', 'input', 'average_runtime', 'speedup', 'efficiency', 'mem_ram',
       'freq_mem', 'freq_cpu', 'mem_l3'],
      dtype='object')

In [5]:
X = df_clean[['cores', 'input', 'average_runtime', 'speedup', 'mem_ram', 'freq_mem', 'freq_cpu', 'mem_l3']].values
y = df_clean['efficiency'].values

### Regression Models

#### MLP Regression Model

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=40)

In [7]:
learning_rate = 0.00001
input_shape = (X_train.shape[1],)

# callback = EarlyStopping(monitor="loss", patience=20)

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(16, activation='relu', input_shape = input_shape))
model.add(tf.keras.layers.Dense(8, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='linear'))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

history = model.fit(X_train, y_train, batch_size=32, epochs=1000, verbose=2, validation_data=(X_test, y_test))

Epoch 1/1000
52/52 - 1s - loss: 226626.4375 - val_loss: 88268.7578 - 544ms/epoch - 10ms/step
Epoch 2/1000
52/52 - 0s - loss: 39183.4336 - val_loss: 10036.0117 - 52ms/epoch - 991us/step
Epoch 3/1000
52/52 - 0s - loss: 3126.6523 - val_loss: 132.4467 - 48ms/epoch - 923us/step
Epoch 4/1000
52/52 - 0s - loss: 53.8220 - val_loss: 111.2295 - 56ms/epoch - 1ms/step
Epoch 5/1000
52/52 - 0s - loss: 44.4687 - val_loss: 99.1871 - 49ms/epoch - 933us/step
Epoch 6/1000
52/52 - 0s - loss: 43.2404 - val_loss: 90.1689 - 49ms/epoch - 938us/step
Epoch 7/1000
52/52 - 0s - loss: 42.0679 - val_loss: 88.0293 - 49ms/epoch - 935us/step
Epoch 8/1000
52/52 - 0s - loss: 41.2599 - val_loss: 73.8928 - 49ms/epoch - 939us/step
Epoch 9/1000
52/52 - 0s - loss: 40.0415 - val_loss: 67.4914 - 56ms/epoch - 1ms/step
Epoch 10/1000
52/52 - 0s - loss: 38.9918 - val_loss: 60.8129 - 49ms/epoch - 934us/step
Epoch 11/1000
52/52 - 0s - loss: 38.1750 - val_loss: 57.2973 - 52ms/epoch - 997us/step
Epoch 12/1000
52/52 - 0s - loss: 37.516

In [8]:
loss = model.evaluate(X_test, y_test)
print(f'Erro no conjunto de teste: {loss}')

predictions = model.predict(X_test)

for i in range(10):
    print(f'Valor Real: {y_test[i]}, Previsão: {predictions[i][0]}')

23/23 [==============================] - 0s 589us/step - loss: 16.4030
Erro no conjunto de teste: 16.40304183959961
23/23 [==============================] - 0s 710us/step
Valor Real: 0.14, Previsão: 0.5264782309532166
Valor Real: 0.19, Previsão: 0.793827474117279
Valor Real: 0.14, Previsão: 0.6834225058555603
Valor Real: 0.07, Previsão: 0.3919796347618103
Valor Real: 0.13, Previsão: 0.8868221640586853
Valor Real: 0.17, Previsão: 0.8450512290000916
Valor Real: 0.13, Previsão: 0.3305019736289978
Valor Real: 0.11, Previsão: 0.7138332724571228
Valor Real: 0.14, Previsão: 0.655674397945404
Valor Real: 0.1, Previsão: 0.3776516318321228


#### MLP Regression Model with Leave-One-Out-Validation (LOOCV)

In [12]:
loo = LeaveOneOut()
mse_scores = []
callback = EarlyStopping(monitor="loss", patience=20)

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(16, activation='relu', input_shape=input_shape))
model.add(tf.keras.layers.Dense(8, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='linear'))


for train_index, test_index in loo.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    learning_rate = 0.001
    input_shape = (X_train.shape[1],)

    model.compile(optimizer='adam', loss='mean_squared_error')

    model.fit(X_train, y_train, epochs=100, verbose=0, callbacks=[callback])

    predictions = model.predict(X_test)

    mse = mean_squared_error(y_test, predictions)
    mse_scores.append(mse)

print(f'Erro médio quadrático usando LOOCV: {min(mse_scores)}')

1/1 [==============================] - 0s 35ms/step
Erro médio quadrático usando LOOCV: 4.475726450791921e-08


In [13]:
PATH_MODEL_SAVE = "model/pascal_model.keras"

model.save(PATH_MODEL_SAVE)